In [ ]:
!pip install wsknn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00


In [ ]:
FNAME = '.../recommender_input.csv'  # uzupełnij ścieżkę do pliku!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Krok 1: przygotowanie danych do modelu

In [ ]:
df = pd.read_csv(FNAME)

In [ ]:
df.columns

Index(['category_id', 'user_id', 'ts'], dtype='object')

In [ ]:
import wsknn

In [ ]:
maps = wsknn.parse_pandas(
    df,
    session_id_key='user_id',
    product_key='category_id',
    time_key='ts'
)

In [ ]:
maps.keys()

dict_keys(['session-map', 'item-map'])

In [ ]:
type(maps['session-map'])

dict

In [ ]:
for key, value in maps['session-map'].items():
    print(key)
    print(value)
    break

1515915625353230683
[[2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415924424278172, 2144415923107266682], [1604911971, 1604912674, 1604912933, 1604913059, 1604913327, 1604913720, 1604913934, 1604913998, 1605696695, 1605696702, 1605697157, 1605697166, 1607769189]]


In [ ]:
for key, value in maps['item-map'].items():
    print(key)
    print(value)
    break

2144415924424278172
[(1515915625519407618, 1515915625467371425, 1515915625519581197, 1515915625433260957, 1515915625517844616, 1515915625518611298, 1515915625519336460, 1515915625519901757, 1515915625519988945, 1515915625520042993, 1515915625468340547, 1515915625520101962, 1515915625520192890, 1515915625520314405, 1515915625520384288, 1515915625520441014, 1515915625520443825, 1515915625475872658, 1515915625412634343, 1515915625520504118, 1515915625520521158, 1515915625520526859, 1515915625520562680, 1515915625520575029, 1515915625506350300, 1515915625515419843, 1515915625520779599, 1515915625520898242, 1515915625485165100, 1515915625520954456, 1515915625520227737, 1515915625520282937, 1515915625519756191, 1515915625521219633, 1515915625509925702, 1515915625521456486, 1515915625521570294, 1515915625521626001, 1515915625521707173, 1515915625521740487, 1515915625520191157, 1515915625521782655, 1515915625521783325, 1515915625521854997, 1515915625521887099, 1515915625448905073, 151591562552

# Krok 2: trening modelu

In [ ]:
model = wsknn.fit(
    sessions=maps['session-map'],
    items=maps['item-map'],
    number_of_recommendations=3,
    return_events_from_session=True,
    recommend_any=True
)

# Krok 3: ewaluacja modelu

In [ ]:
sample_sessions = [v[1] for idx, v in enumerate(maps['session-map'].items()) if idx < 1000]

len(sample_sessions)

1000

In [ ]:
sample_sessions[0]

[[2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415924424278172,
  2144415923107266682],
 [1604911971,
  1604912674,
  1604912933,
  1604913059,
  1604913327,
  1604913720,
  1604913934,
  1604913998,
  1605696695,
  1605696702,
  1605697157,
  1605697166,
  1607769189]]

In [ ]:
scoring = wsknn.score_model(
    sessions=sample_sessions,
    trained_model=model
)

In [ ]:
scoring

{'MRR': 0.8400462962962963,
 'Precision': 0.32546296296296295,
 'Recall': 0.36492452471568665}

# Krok 4: przykładowe rekomendacje i powrót do początku

In [ ]:
sample_sessions = [{v[0]: v[1]} for idx, v in enumerate(maps['session-map'].items()) if idx < 100]

recs = wsknn.batch_predict(
    model=model,
    sessions=sample_sessions
)

In [ ]:
recs[0]

{1515915625353230683: [(2144415923107266682, 69.46153846153855),
  (2144415926932472027, 1.6153846153846154),
  (2144415922427789416, 1.0769230769230769)]}